In [1]:
#Habilitar intellisense
%config IPCompleter.greedy=True

In [39]:
import pandas as pd
import numpy as np
import nltk
from xgboost.sklearn import XGBClassifier
import string
from nltk.tokenize import sent_tokenize
from bs4 import BeautifulSoup


In [12]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\BERNANRDO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\BERNANRDO\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [5]:
data = pd.read_csv('imdb_dataset.csv',encoding="ISO-8859-1")
data.tail()

,Review,Label
49995,"Seeing as the vote average was pretty low, and...",pos
49996,"The plot had some wretched, unbelievable twist...",pos
49997,I am amazed at how this movie(and most others ...,pos
49998,A Christmas Together actually came before my t...,pos
49999,Working-class romantic drama from director Mar...,pos


In [6]:
#palabras = nltk.word_tokenize(data["Review"])
#palabras
data["Review"]

0        Once again Mr. Costner has dragged out a movie...
1        This is an example of why the majority of acti...
2        First of all I hate those moronic rappers, who...
3        Not even the Beatles could write songs everyon...
4        Brass pictures (movies is not a fitting word f...
                               ...                        
49995    Seeing as the vote average was pretty low, and...
49996    The plot had some wretched, unbelievable twist...
49997    I am amazed at how this movie(and most others ...
49998    A Christmas Together actually came before my t...
49999    Working-class romantic drama from director Mar...
Name: Review, Length: 50000, dtype: object

In [9]:
#data.sample(frac=0.1).shape
#pd.DataFrame()


data_prueba=data.sample(frac=0.2)
palabrasParada = set( nltk.corpus.stopwords.words('english') + list(string.punctuation))



In [10]:
def borrarSiContiene(palabra, borrar):
    if borrar in palabra:
        return 
    return palabra
def Tokenizar(texto):
    texto=texto.lower()
    sinHTML=BeautifulSoup(texto).text
    #sinHTML=remove_html_tags(texto)#quita html
    palabras = nltk.word_tokenize(sinHTML)#separa las palabras
    return [palabra for palabra in palabras if palabra not in palabrasParada]#quita stopwords


#data_prueba2=data_prueba.apply(lambda x: pd.Series(Tokenizar("sdasd asd a"),index=["Tokenizado"]),axis=1)
#data_prueba2.head()
# #df.apply(lambda x: pd.Series([1, 2], index=['foo', 'bar']), axis=1)

#df['CustomRating'] = df.apply(lambda x: custom_rating(x['Genre'],x['Rating']),axis=1)


In [13]:
def listToString(s):  
    str1 = " " 
    return (str1.join(s)) 

listaDeComentarios=[]
for index, row in data_prueba.iterrows():
    a=Tokenizar(row['Review'])
    x=listToString(a)  
    z={'tokenizado':x,'clasificacion':row['Label']}
    listaDeComentarios.append(z)



In [14]:
listaDeSoloComentarios=[]
for index, row in data_prueba.iterrows():
    a=Tokenizar(row['Review'])
    x=listToString(a)  
    listaDeSoloComentarios.append(x)

In [15]:
listaDeSoloComentarios[0]

"avoid film looking entertainment.it filled wannabes trying something emraan wasted role tour guide falls newcomer needs go acting school seriously get people 're pretty n't mean act actress.asmit patel needs send apology letter everyone accidentally watches makes fool poor excuse film plays insipid wannabe gangster drugs girls forces fall love sells highest bidder"

In [16]:
def get_problem_vocabulary(normalized_corpus):
    all_tokens = [] 
    for document in normalized_corpus:
        all_tokens.extend(document.split())  
    #[all_tokens.extend(document.split()) for document in normalized_corpus]
    all_tokens_sorted = sorted(set(all_tokens))
    
    token_and_position = {}
    for i, token in enumerate(all_tokens_sorted):
        token_and_position[token] = i
    
    return token_and_position
    
    
    

    
problem_vocabulary = get_problem_vocabulary(listaDeSoloComentarios)
#len(problem_vocabulary)


In [17]:

(len(problem_vocabulary))

77376

In [18]:
def one_hot_vector(document, problem_vocabulary):
    vector = np.zeros(len(problem_vocabulary),dtype=int)
    for token in document.split():
        vector[problem_vocabulary[token]] = 1
    
    return vector
def one_hot_matriz(lista):
    matriz=[]
    for i in lista:
        matriz.append(one_hot_vector(i, problem_vocabulary))
    return matriz
        
matriz=one_hot_matriz(listaDeSoloComentarios)


In [19]:
print(len(data_prueba))

10000


In [20]:
matrizDic=[]
i=0
for index, row in data_prueba.iterrows():
    
    z = {'tokenizado':matriz[i],'clasificacion':row['Label']}
    i=i+1
    matrizDic.append(z)


In [21]:
from random import sample 

muestra=sample(matrizDic,3000)


In [22]:
listaParaEntrenar=[]
for i in muestra:
    if(i["clasificacion"]=="pos"):
        
        listaParaEntrenar.append([i["tokenizado"], 1])
       
    else:
        listaParaEntrenar.append([i["tokenizado"], 0])

In [23]:

df = pd.DataFrame(listaParaEntrenar)
df

,0,1
0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
2,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
3,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
...,...,...
2995,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
2996,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
2997,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
2998,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1


In [24]:
import os
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

In [25]:
df3 = df[0].apply(pd.Series)
df3["res"]=df[1]
    

In [26]:
df3

,0,1,2,3,4,5,6,7,8,9,...,77367,77368,77369,77370,77371,77372,77373,77374,77375,res
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2997,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2998,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [27]:

X = df3.iloc[:, np.r_[0:77376]]
y = df3["res"] # Target variable

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training, 30% test
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(2100, 77376) (900, 77376) (2100,) (900,)


In [29]:
treev1 = DecisionTreeClassifier()
treev1.fit(X_train,y_train )

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [32]:
# métricas de desempeño
# accuracy
print('accuracy del clasificador - version 1 : {0:.2f}'.format(accuracy_score(y_test, treev1.predict(X_test))))
# confusion matrix
print('matriz de confusión del clasificador - version 1: \n {0}'.format(confusion_matrix(y_test, treev1.predict(X_test))))
# precision 
print('precision del clasificador - version 1 : {0:.2f}'.format(precision_score(y_test, treev1.predict(X_test))))
# recall
print('recall del clasificador - version 1 : {0:.2f}'.format(recall_score(y_test, treev1.predict(X_test))))
# f1
print('f1 del clasificador - version 1 : {0:.2f}'.format(f1_score(y_test, treev1.predict(X_test))))

accuracy del clasificador - version 1 : 0.69
matriz de confusión del clasificador - version 1: 
 [[292 165]
 [115 328]]
precision del clasificador - version 1 : 0.67
recall del clasificador - version 1 : 0.74
f1 del clasificador - version 1 : 0.70


In [40]:
treev2 = XGBClassifier(n_stimators=100)
treev2.fit(X_train,y_train )

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              n_stimators=100, nthread=None, objective='binary:logistic',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=None, silent=None, subsample=1, verbosity=1)

In [42]:
# métricas de desempeño
# accuracy
print('accuracy del clasificador - version 1 : {0:.2f}'.format(accuracy_score(y_test, treev2.predict(X_test))))
# confusion matrix
print('matriz de confusión del clasificador - version 1: \n {0}'.format(confusion_matrix(y_test, treev2.predict(X_test))))
# precision 
print('precision del clasificador - version 1 : {0:.2f}'.format(precision_score(y_test, treev2.predict(X_test))))
# recall
print('recall del clasificador - version 1 : {0:.2f}'.format(recall_score(y_test, treev2.predict(X_test))))
# f1
print('f1 del clasificador - version 1 : {0:.2f}'.format(f1_score(y_test, treev2.predict(X_test))))

accuracy del clasificador - version 1 : 0.78
matriz de confusión del clasificador - version 1: 
 [[324 133]
 [ 61 382]]
precision del clasificador - version 1 : 0.74
recall del clasificador - version 1 : 0.86
f1 del clasificador - version 1 : 0.80


In [47]:
space ={
    'n_estimators':hp.quniform('n_estimators',10,100,1), #probar con valores entre 100 - 100, con incrementos de 1 
    'learning_rate':hp.quniform('learning_rate',0.025,0.5,0.025),
    'max_depth':hp.quniform('max_depth',1,13,1),
    'subsample': hp.quniform('subsample',0.5,1,0.05),
    'colsample_bytree':hp.quniform('colsample_bytree',0.5,1,0.05),
    'nthread':3, #cuando se posible, paralelizar el procesamiento empleando hasta 6 hilos
    'silent':1
}

In [48]:
def objective(params):
    params['n_estimators']=int(params['n_estimators'])
    params['max_depth']=int(params['max_depth'])  
    clf=XGBClassifier(**params) #https://treyhunner.com/2018/10/asterisks-in-python-what-they-are-and-how-to-use-them/
    clf.fit(X_train,y_train)   
    accuracy = accuracy_score(y_test, clf.predict(X_test))
    return {'loss':1-accuracy,'status':STATUS_OK}

In [44]:
from hyperopt import fmin, tpe, hp, STATUS_OK,Trials

In [45]:
trials=Trials()

In [49]:
best=fmin(objective,space,algo=tpe.suggest,trials=trials,max_evals=10)

100%|███████████████████████████████████████████████| 10/10 [23:35<00:00, 141.52s/trial, best loss: 0.1844444444444444]


In [50]:
print(best)

{'colsample_bytree': 0.8, 'learning_rate': 0.07500000000000001, 'max_depth': 12.0, 'n_estimators': 88.0, 'subsample': 0.9500000000000001}


In [51]:
tree_v3 = XGBClassifier(**best)
tree_v3

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0,
              learning_rate=0.07500000000000001, max_delta_step=0,
              max_depth=12.0, min_child_weight=1, missing=None,
              n_estimators=88.0, n_jobs=1, nthread=None,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
              subsample=0.9500000000000001, verbosity=1)

In [55]:
tree_v3.fit(X_train,y_train)

TypeError: 'float' object cannot be interpreted as an integer

In [53]:
print('accuracy del clasificador - version 1 : {0:.2f}'.format(accuracy_score(y_test, tree_v3.predict(X_test))))
# confusion matrix
print('matriz de confusión del clasificador - version 1: \n {0}'.format(confusion_matrix(y_test, tree_v3.predict(X_test))))
# precision 
print('precision del clasificador - version 1 : {0:.2f}'.format(precision_score(y_test, tree_v3.predict(X_test))))
# recall
print('recall del clasificador - version 1 : {0:.2f}'.format(recall_score(y_test, tree_v3.predict(X_test))))
# f1
print('f1 del clasificador - version 1 : {0:.2f}'.format(f1_score(y_test, tree_v3.predict(X_test))))

XGBoostError: need to call fit or load_model beforehand

In [31]:
muestra[0]

{'tokenizado': array([1, 0, 0, ..., 0, 0, 0]), 'clasificacion': 'pos'}

In [58]:
from sklearn.ensemble import RandomForestClassifier
treeV4=RandomForestClassifier(n_estimators=88)

In [59]:
treeV4.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=88,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [60]:
print('accuracy del clasificador - version 1 : {0:.2f}'.format(accuracy_score(y_test, treeV4.predict(X_test))))
# confusion matrix
print('matriz de confusión del clasificador - version 1: \n {0}'.format(confusion_matrix(y_test, treeV4.predict(X_test))))
# precision 
print('precision del clasificador - version 1 : {0:.2f}'.format(precision_score(y_test, treeV4.predict(X_test))))
# recall
print('recall del clasificador - version 1 : {0:.2f}'.format(recall_score(y_test, treeV4.predict(X_test))))
# f1
print('f1 del clasificador - version 1 : {0:.2f}'.format(f1_score(y_test, treeV4.predict(X_test))))

accuracy del clasificador - version 1 : 0.82
matriz de confusión del clasificador - version 1: 
 [[362  95]
 [ 67 376]]
precision del clasificador - version 1 : 0.80
recall del clasificador - version 1 : 0.85
f1 del clasificador - version 1 : 0.82
